In [18]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

In [19]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [21]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'GYV6NG0EXYH6D3AV',
  'HostId': '0Hlj7t2cGGRNUGhN8PcsGW7kVaqoSrDNudc0q/t1IetxF7erK8JPO8LJ/hjV8IFcsVCLDDHEfCw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '0Hlj7t2cGGRNUGhN8PcsGW7kVaqoSrDNudc0q/t1IetxF7erK8JPO8LJ/hjV8IFcsVCLDDHEfCw=',
   'x-amz-request-id': 'GYV6NG0EXYH6D3AV',
   'date': 'Sat, 17 Jan 2026 21:43:10 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc()),
   'BucketArn': 'arn:aws:s3:::aft-sandbox-540362055257'},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc()),
   'BucketArn': 'arn:aws:s3:::amplify-citiesindicatorsapi-dev-10508-deployment'},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc

In [22]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=2,
    package_sync_ignore=["pyspark", "pypandoc"],
    worker_options={"nthreads": 1}
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2026-01-17 16:43:09,669][INFO    ][coiled] Fetching latest package priorities...
[2026-01-17 16:43:09,670][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2026-01-17 16:43:10,096][INFO    ][coiled.package_sync] Scanning 446 conda packages...
[2026-01-17 16:43:10,101][INFO    ][coiled.package_sync] Scanning 261 python packages...
[2026-01-17 16:43:10,583][INFO    ][coiled] Running pip check...
[2026-01-17 16:43:11,051][INFO    ][coiled] Validating environment...
[2026-01-17 16:43:11,686][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2026-01-17 16:43:11,691][INFO    ][coiled] Creating wheel for /opt/spark-2.2.0/python...
[2026-01-17 16:43:11,697][WARNING ][coiled.package_sync] Package - debugpy, debugpy~=1.8.16 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2026-01-17 16:43:11,697][INFO    ][coiled.package_sync] Package - aiobotocore, Pip check had the follow

Started a new Dask client on Coiled. Dashboard is available at https://cluster-rqtaa.dask.host/-c1C3tvp9cvQvzcz/status


/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py:1590: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.4.4  | 4.4.5     | 4.4.5   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


2026-01-17 17:08:30,022 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [23]:
import s3fs
import fsspec
import traceback

import os
import os
import json
import time
import uuid
import socket
import traceback
from datetime import datetime, timezone

import pandas as pd
from cloudpathlib import S3Path
from gather_data_cities import *

fs = s3fs.S3FileSystem(anon=False)
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities = [x.split('/')[-1] for x in search_buffer_files]
print(len(cities))
cities =cities[1:]
print(len(cities))

1239
1238


In [24]:
import os
import io
import time
import json
import traceback
import contextlib
from datetime import datetime

import pandas as pd
import s3fs
from cloudpathlib import S3Path
from dask.distributed import Semaphore

# You already have these constants in gather_data_cities.py, but import them or redefine:
# MAIN_PATH, INPUT_PATH, OUTPUT_PATH, OUTPUT_PATH_CSV, SEARCH_BUFFER_PATH, ROADS_PATH, INTERSECTIONS_PATH, NATURAL_FEATURES_PATH, BUILDINGS_PATH

fs = s3fs.S3FileSystem(anon=False)

RUN_ID = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
LOGS_S3_DIR = f"{OUTPUT_PATH}/logs/gather_data/{RUN_ID}"   # e.g. s3://.../output/logs/gather_data/20260115T....
SUMMARY_S3_PATH = f"{LOGS_S3_DIR}/summary.csv"

# ---- IMPORTANT: throttle Overpass concurrency ----
# Overpass + AWS + 10 workers will often explode without this.
# Try max_leases=2 first; if still unstable, use 1.
overpass_sem = Semaphore(max_leases=2, name="overpass")


def _s3_exists(path: str) -> bool:
    try:
        return fs.exists(path)
    except Exception:
        return False

def _expected_outputs(city_clean: str) -> dict:
    """Return expected output paths + existence flags."""
    roads_path = f"{ROADS_PATH}/{city_clean}/{city_clean}_OSM_roads.geoparquet"
    inter_path = f"{INTERSECTIONS_PATH}/{city_clean}/{city_clean}_OSM_intersections.geoparquet"
    nat_path   = f"{NATURAL_FEATURES_PATH}/{city_clean}/{city_clean}_OSM_natural_features_and_railroads.geoparquet"

    # Overture output path depends on your CLI output. In your code it's:
    # os.path.join(BUILDINGS_PATH, city_clean) and filename Overture_building_<city>.geoparquet
    bld_path   = f"{BUILDINGS_PATH}/{city_clean}/Overture_building_{city_clean}.geoparquet"

    return {
        "roads_path": roads_path,
        "intersections_path": inter_path,
        "natural_path": nat_path,
        "buildings_path": bld_path,
        "exists_roads": _s3_exists(roads_path),
        "exists_intersections": _s3_exists(inter_path),
        "exists_natural": _s3_exists(nat_path),
        "exists_buildings": _s3_exists(bld_path),
    }

def gather_data_city_logged(city_name: str) -> dict:
    from gather_data_cities import gather_data_city

    city_clean = city_name.replace(" ", "_")
    t0 = time.time()

    lines = []
    def log(msg: str):
        lines.append(f"[{city_clean}] {msg}")

    result = {"city": city_clean, "status": None, "error": "", "elapsed_s": None, "log_s3": ""}

    try:
        log("START")
        res = gather_data_city(city_clean)
        log(f"gather_data_city returned type={res.get('type')} status={res.get('status')}")
        result["gather_return_type"] = res.get("type", "")
        result["gather_return_status"] = res.get("status", "")

        outs = _expected_outputs(city_clean)
        result.update(outs)

        result["status"] = "OK" if result["exists_roads"] and result["exists_intersections"] else "PARTIAL"

    except Exception as e:
        result["status"] = "FAIL"
        result["error"] = "".join(traceback.format_exception(None, e, e.__traceback__))
        log(f"EXCEPTION: {e}")

        try:
            result.update(_expected_outputs(city_clean))
        except Exception:
            pass

    finally:
        result["elapsed_s"] = round(time.time() - t0, 2)
        log(f"END elapsed_s={result['elapsed_s']} status={result['status']}")

        # Upload log text (only this city’s lines)
        log_text = "\n".join(lines) + "\n"
        local_log = f"/tmp/{city_clean}_{RUN_ID}.log"
        with open(local_log, "w", encoding="utf-8", errors="replace") as f:
            f.write(log_text)

        log_s3 = f"{LOGS_S3_DIR}/city_logs/{city_clean}.log"
        S3Path(log_s3).parent.mkdir(parents=True, exist_ok=True)
        S3Path(log_s3).upload_from(local_log)
        result["log_s3"] = log_s3

    return result



/var/folders/nn/3mdkp6sx1n3d955f3wqgdbb00000gn/T/ipykernel_83379/1327798151.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  RUN_ID = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")


In [25]:



def utc_now_iso():
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")


def make_run_id(prefix="ils"):
    return f"{prefix}-{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}-{uuid.uuid4().hex[:8]}"


def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)
    return path


def write_text(path: str, text: str):
    ensure_dir(os.path.dirname(path))
    with open(path, "a", encoding="utf-8") as f:
        f.write(text)


def safe_s3_upload(local_path: str, s3_uri: str):
    # cloudpathlib handles mkdir + upload nicely
    s3p = S3Path(s3_uri)
    s3p.parent.mkdir(parents=True, exist_ok=True)
    s3p.upload_from(local_path)
    return s3_uri


In [26]:
import os, time, socket, traceback
from datetime import datetime, timezone

def utc_now():
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

def append_log(path: str, msg: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "a", encoding="utf-8") as f:
        f.write(msg)

def run_city(city_name: str, run_id: str, log_dir: str, fs) -> dict:
    city_clean = city_name.replace(" ", "_").strip()
    host = socket.gethostname()
    log_path = os.path.join(log_dir, f"{city_clean}.log")

    append_log(log_path, f"[{utc_now()}] START run_id={run_id} city={city_clean} host={host}\n")

    # ---- SKIP logic (fast) ----
    done, exists_map = is_city_done(city_clean, fs)
    append_log(log_path, f"[{utc_now()}] CHECK done={done} exists={exists_map}\n")

    if done:
        append_log(log_path, f"[{utc_now()}] SKIP already_done\n")
        return {
            "run_id": run_id,
            "city": city_clean,
            "status": "SKIP",
            "elapsed_s": 0.0,
            "host": host,
            "exists_roads": exists_map["roads"],
            "exists_intersections": exists_map["intersections"],
            "exists_natural": exists_map["natural"],
            "error": "",
        }

    # ---- Run gather ----
    t0 = time.time()
    try:
        result = gather_data_city(city_clean)  # your function returns dict

        elapsed = time.time() - t0

        # re-check outputs after run
        done2, exists_map2 = is_city_done(city_clean, fs)

        append_log(log_path, f"[{utc_now()}] DONE_CALL gather_result={result}\n")
        append_log(log_path, f"[{utc_now()}] POSTCHECK done={done2} exists={exists_map2}\n")
        append_log(log_path, f"[{utc_now()}] OK elapsed_s={elapsed:.2f}\n")

        status = "OK" if done2 else "PARTIAL"

        return {
            "run_id": run_id,
            "city": city_clean,
            "status": status,
            "elapsed_s": round(elapsed, 3),
            "host": host,
            "exists_roads": exists_map2["roads"],
            "exists_intersections": exists_map2["intersections"],
            "exists_natural": exists_map2["natural"],
            "error": "" if status != "PARTIAL" else "Some expected outputs missing after run",
        }

    except Exception as e:
        elapsed = time.time() - t0
        tb = "".join(traceback.format_exception(type(e), e, e.__traceback__))
        append_log(log_path, f"[{utc_now()}] FAIL elapsed_s={elapsed:.2f}\n{tb}\n")

        return {
            "run_id": run_id,
            "city": city_clean,
            "status": "FAIL",
            "elapsed_s": round(elapsed, 3),
            "host": host,
            "exists_roads": False,
            "exists_intersections": False,
            "exists_natural": False,
            "error": str(e)[:2000],
        }

def expected_outputs(city_clean: str):
    return {
        "roads": f"{ROADS_PATH}/{city_clean}/{city_clean}_OSM_roads.geoparquet",
        "intersections": f"{INTERSECTIONS_PATH}/{city_clean}/{city_clean}_OSM_intersections.geoparquet",
        "natural": f"{NATURAL_FEATURES_PATH}/{city_clean}/{city_clean}_OSM_natural_features_and_railroads.geoparquet",
        # buildings: not reliably on S3 with current code
    }

def is_city_done(city_clean: str, fs) -> tuple[bool, dict]:
    paths = expected_outputs(city_clean)
    exists = {k: fs.exists(p) for k, p in paths.items()}
    done = all(exists.values())
    return done, exists



In [27]:
cities_all = pd.read_csv('../data/city_lists/cities_ssa_latam.csv',sep=';')

In [28]:
len(cities)

1238

In [29]:
from getting_1000_cities_grids import *



all_cities_names = [s3_safe_token(x) for x in cities_all['city_name']]
all_countries_names = [s3_safe_token(x) for x in cities_all['country_name']]

all_cities = [f"{a}__{b}" for a, b in zip(all_cities_names, all_countries_names)]


In [30]:
search_buffer_files = fs.ls(ROADS_PATH)

existing_features_cities = [x.split('/')[-1] for x in search_buffer_files]

missing_cities = set(all_cities).difference(set(existing_features_cities))
print(len(missing_cities))
#print(missing_cities)

5


In [31]:
missing_cities

{'Bugama__Nigeria',
 'Mexico_City__Mexico',
 'Mubi__Nigeria',
 'Ngo__Nigeria',
 'San_Pedro_de_Macoris__Dominican_Republic'}

In [ ]:
#missing_cities_before = pd.read_csv('missing_cities_before.csv')
#set(missing_cities_before['0']).difference(set(missing_cities))

In [32]:
import dask.bag as db
import pandas as pd
from cloudpathlib import S3Path

bag = db.from_sequence(missing_cities, partition_size=1)
records = bag.map(gather_data_city_logged).compute()

df = pd.DataFrame(records)
print(df["status"].value_counts(dropna=False))

local_summary = f"/tmp/summary_{RUN_ID}.csv"
df.to_csv(local_summary, index=False, sep=';')

S3Path(LOGS_S3_DIR).mkdir(parents=True, exist_ok=True)
S3Path(SUMMARY_S3_PATH).upload_from(local_summary)

print("✅ Summary written to:", SUMMARY_S3_PATH)


status
PARTIAL    5
Name: count, dtype: int64
✅ Summary written to: s3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/logs/gather_data/20260117T214509Z/summary.csv


In [ ]:
'''
import dask.bag as db


#cities = ["Accra", "Abidjan", "Bamako", "Bogota", "Belo_Horizonte", "Campinas", "Cape_Town", "Lagos",  "Nairobi", "Luanda"] #, Maputo, "Mogadishu",

bag = db.from_sequence(cities, partition_size=1)
results = bag.map(gather_data_city).compute()
'''


In [ ]:
'''
from pre_processing import *
cities = ["Belo_Horizonte", "Campinas", "Cape_Town", "Lagos",  "Nairobi", "Luanda"] #, Maputo, "Mogadishu", #"Accra", "Abidjan", "Bamako", "Bogota", 
for city_name in cities:
    produce_blocks(city_name,YOUR_NAME).compute()
'''

In [ ]:
def check_creds():
    import os
    return {
        "AWS_ACCESS_KEY_ID": bool(os.getenv("AWS_ACCESS_KEY_ID")),
        "AWS_SECRET_ACCESS_KEY": bool(os.getenv("AWS_SECRET_ACCESS_KEY")),
        "AWS_SESSION_TOKEN": bool(os.getenv("AWS_SESSION_TOKEN")),
        "AWS_DEFAULT_REGION": os.getenv("AWS_DEFAULT_REGION"),
    }

CLIENT.run(check_creds)